In [43]:
import os
import pandas as pd
import numpy as np

In [44]:
#Set notebook preferences
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [45]:
#Load modules
os.chdir('/Users/ksharma/Documents/ML Engineer/Machine Learning/Projects/paysim_credit_fraud_analysis/')
from src.data.import_export_data import load_config

#Import data
path= r'/Users/ksharma/Documents/ML Engineer/Machine Learning/Projects/paysim_credit_fraud_analysis/'
config_name= 'config.yaml'

config= load_config(config_name= config_name, path=path)

dtypes= {'isFraud':'bool', 'step':'object'}
rawData= pd.read_csv(config['paths']['cleanedData'] + 'processedData.csv', dtype= dtypes, index_col= [0])

/Users/ksharma/miniforge3/envs/TensorFlow/lib/python3.8/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


**Data Overview**

In [46]:
display(rawData.shape)
display(rawData.head())

(6362620, 6)

,amount,isFraud,oldbalanceDest,oldbalanceOrg,step,type
0,9839.640,False,0.000,170136.000,1,PAYMENT
1,1864.280,False,0.000,21249.000,1,PAYMENT
2,181.000,True,0.000,181.000,1,TRANSFER
3,181.000,True,21182.000,181.000,1,CASH_OUT
4,11668.140,False,0.000,41554.000,1,PAYMENT


In [47]:
rawData.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6362620 entries, 0 to 6362619
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   amount          float64
 1   isFraud         bool   
 2   oldbalanceDest  float64
 3   oldbalanceOrg   float64
 4   step            object 
 5   type            object 
dtypes: bool(1), float64(3), object(2)
memory usage: 297.3+ MB


**Preprocess Data**

In [48]:
#seperate target from data
X= rawData.drop(columns= 'isFraud', axis= 1)
y= rawData.loc[:, 'isFraud']

In [49]:
#Subset numeric features
numeric_features= list(X.select_dtypes(include='float64').columns)
numeric_features

['amount', 'oldbalanceDest', 'oldbalanceOrg']

In [50]:
#Create numeric transformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

num_transformer= StandardScaler()

In [51]:
#subset categorical features
cat_features= list(X.select_dtypes(exclude='float64').columns)
cat_features


['step', 'type']

In [52]:
#Create cat transformer
cat_transformer= OneHotEncoder(handle_unknown= 'ignore')

In [53]:
preprocessor= ColumnTransformer(transformers=[('num', num_transformer, numeric_features),
                                             ('cat', cat_transformer, cat_features)])

In [54]:
#init models to try

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

baseModels= config['machineLearning']['baseModels']
baseModels

['LogisticRegression()',
 'RandomForestClassifier()',
 'GradientBoostingClassifier()',
 'AdaBoostClassifier()',
 'KNeighborsClassifier()',
 'GaussianNB()']

In [ ]:
names= ['LogisticRegression',
 'RandomForestClassifier',
 'GradientBoostingClassifier',
 'AdaBoostClassifier',
 'KNeighborsClassifier',
 'GaussianNB']

In [59]:
from sklearn.metrics import confusion_matrix, fbeta_score, make_scorer

scoring= {'Recall':'recall', 'F2': make_scorer(fbeta_score, beta= 2)}

In [61]:
from src.models.modeling import Data, Models


**Test base models on baseline data (No over/under sampling applied)**

In [66]:
j= Data(X, y)
sample1.sample_data(n= 100_000)

,amount,oldbalanceDest,oldbalanceOrg,step,type
3966102,7736.520,0.000,12157.000,297,PAYMENT
5279478,191702.290,121557.100,104127.000,372,CASH_IN
850386,400455.250,0.000,100756.000,41,CASH_OUT
4623906,21959.550,51011.000,0.000,329,CASH_OUT
899604,9514.810,489737.050,29976.000,42,CASH_OUT
...,...,...,...,...,...
5437976,214248.640,6580100.980,4578980.800,378,CASH_IN
2202629,6591.860,0.000,0.000,185,PAYMENT
5788097,209724.430,374881.240,0.000,401,TRANSFER
1988139,204933.590,7610550.990,1619742.620,179,CASH_IN


In [68]:
sample1.data.describe()

,amount,oldbalanceDest,oldbalanceOrg
count,6362620.000,6362620.000,6362620.000
mean,179861.904,1100701.667,833883.104
std,603858.231,3399180.113,2888242.673
min,0.000,0.000,0.000
25%,13389.570,0.000,0.000
50%,74871.940,132705.665,14208.000
75%,208721.478,943036.708,107315.175
max,92445516.640,356015889.350,59585040.370
